New notebook to test the difference between the tech rec mongo db and gal alum.
Currently, code working for gal alum but not tech recruiters.
1. Copy progress from mongodb_test
2. Replicate the gal alum process from
* Connection to mongo db
* Pull data into df
* Manipulate to correct format
* Create csv of all companies
* Create csvs of individual companies

3. New notebook section for tech recruiters

In [15]:
import pandas as pd
import mongo
import ast

# Galvanize Alumni CSV build

In [2]:
mongo.connect_mongo()
mongo.connect_coll('gal_part_proj', 'gal_alum')
cursor = mongo.coll.find({}, {'_id':0})
df = pd.DataFrame(cursor)
df = df.T

In [3]:
index = list()
results = list()

for idx, row in df.iterrows():
    index.append(idx)
    results.append(row[row.notnull()].values[0])

clean_df = pd.DataFrame(results, index=index)
clean_df.rename({0: 'Scrape Results'}, axis=1, inplace=True)
clean_df.head()

,Scrape Results
University of Texas at Austin,People Link 404
Apple,{'Erin Gong': 'https://www.linkedin.com/in/eri...
Dell Technologies,{'Daniel Donovan': 'https://www.linkedin.com/i...
IBM,{'James Helfrich': 'https://www.linkedin.com/i...
National Instruments,No results


# Austin Technical Recruiters CSV build
## Load data from mongo

In [4]:
mongo.connect_mongo()
mongo.connect_coll('gal_part_proj', 'ATX_tech_rec')
cursor = mongo.coll.find({}, {'_id':0})

df_tr = pd.DataFrame(cursor)
df_tr = df_tr.T

## Austin Tech Recruiters pipeline error corrections
My dataframe pipeline is failing for the tech recruiters mongo pull. Successively passing the Alumni pull with the same process. Steps to troubleshoot proceed.

In [5]:
# pipeline for ATX Tech Recruiters
index_tr = list()
results_tr = list()

for idx, row in df_tr.iterrows():
    index_tr.append(idx)
    results_tr.append(row[row.notnull()].values[0])

results_tr.insert(0, 'ignore')
index_tr.insert(0, 'ignore')    
clean_df_tr = pd.DataFrame(results_tr, index=index_tr)
clean_df_tr.drop('ignore', axis=0, inplace=True)
clean_df_tr.rename({0: 'Scrape Results'}, axis=1, inplace=True)
clean_df_tr.head()

,Scrape Results
9Gauge Partners,{'Carrie Guillory': 'https://www.linkedin.com/...
Abbott,{'Brandy Williams Broden': 'https://www.linked...
Aceable,"{'Andrew Bolisay (he, him, his)': 'https://www..."
AdbaKx,Company Page 404
Addison Group,{'Hattie Cox': 'https://www.linkedin.com/in/ha...


# Convert DataFrame to CSVs for each company (observations)

In [12]:
# change below assignment for saving different CSVs
contacts = clean_df_tr #clean_df
employee_type = 'tech_rec' #alumni

In [10]:
dict_flag = contacts['Scrape Results'].apply(lambda x: isinstance(x, dict))

In [11]:
dict_flag

9Gauge Partners     True
Abbott              True
Aceable             True
AdbaKx             False
Addison Group       True
                   ...  
XO Group           False
Xtivia              True
Z5 Inventory       False
Zilliant            True
Zuar               False
Name: Scrape Results, Length: 197, dtype: bool

In [16]:
dict_flag = contacts['Scrape Results'].apply(lambda x: isinstance(x, dict))
dict_flag = contacts['Scrape Results'].apply(lambda x: ast.literal_eval(x))
contacts = clean_df_tr[dict_flag] #clean_df
employee_type = 'tech_rec' #alumni

for i, c in zip(contacts.index, contacts):
    co_contacts = pd.DataFrame(c.values(), index=c.keys())
    co_contacts.to_csv('../data/{0}/{1}.csv'.format(employee_type, i))
    break

ValueError: malformed node or string: {'Carrie Guillory': 'https://www.linkedin.com/in/carrieguillory/'}

# gDrive interaction testing

In [111]:
import gspread